# Load module data based on gephi's co-membership network

In [ ]:
import pandas as pd
ds = pd.read_csv(r"C:\Users\lunad\Desktop\hai\all_ds_co membership.csv")

In [ ]:
# Filter the main 9 modules
modularity_classes = [0,1,2,3,4,5,6,7,8]
ds = ds[ds.modularity_class.isin(modularity_classes)]

# Create subsets for each module

In [ ]:
module_1 = ds[ds['modularity_class']== 1]['Id'].tolist()
module_2 = ds[ds['modularity_class']== 2]['Id'].tolist()
module_3 = ds[ds['modularity_class']== 3]['Id'].tolist()
module_4 = ds[ds['modularity_class']== 4]['Id'].tolist()
module_5 = ds[ds['modularity_class']== 5]['Id'].tolist()
module_6 = ds[ds['modularity_class']== 6]['Id'].tolist()
module_7 = ds[ds['modularity_class']== 7]['Id'].tolist()
module_8 = ds[ds['modularity_class']== 8]['Id'].tolist()
module_0 = ds[ds['modularity_class']== 0]['Id'].tolist()

# Load group info data

In [ ]:
import json as js
with open(r"C:\Users\lunad\Desktop\flickr\group_info\New folder\ginfo_clean_ver.json") as file_input:
        c_ginfo = js.load(file_input)
        group_id = list(c_ginfo.keys())
        #c_ginfo = list(c_ginfo.values())

In [ ]:
with open(r"C:\Users\lunad\Desktop\flickr\group_info\New folder\sorted_ginfo.json") as file_input:
        sorted_ginfo = js.load(file_input)
        group_id1 = list(sorted_ginfo.keys())
        sorted_ginfo = list(sorted_ginfo.values())

In [ ]:
# Assign group info for each module 
ginfo_module_2 = [c_ginfo[i] for i in module_2 if i in c_ginfo]
ginfo_module_3 = [c_ginfo[i] for i in module_3 if i in c_ginfo]
ginfo_module_4 = [c_ginfo[i] for i in module_4 if i in c_ginfo]
ginfo_module_5 = [c_ginfo[i] for i in module_5 if i in c_ginfo]
ginfo_module_6 = [c_ginfo[i] for i in module_6 if i in c_ginfo]
ginfo_module_7 = [c_ginfo[i] for i in module_7 if i in c_ginfo]
ginfo_module_8 = [c_ginfo[i] for i in module_8 if i in c_ginfo]
ginfo_module_0 = [c_ginfo[i] for i in module_0 if i in c_ginfo]
ginfo_module_1 = [c_ginfo[i] for i in module_1 if i in c_ginfo]

In [ ]:
# Combine them
ginfo_module = [ginfo_module_2,ginfo_module_1,ginfo_module_3,ginfo_module_5,ginfo_module_6,ginfo_module_7,
             ginfo_module_8,ginfo_module_0]
module_name = ["module2","module1","module3","module5","module6","module7","module8","module0"]

# Cleaning text for the group info

In [ ]:
import string
from sklearn.preprocessing import MultiLabelBinarizer
import nltk
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer()

ascii_chars = set(string.printable)  # speeds things up
def remove_non_ascii_prinatble_from_list(list_of_words):
    return [word for word in list_of_words 
            if all(char in ascii_chars for char in word)]

clean_ginfo_module = []
for a in range(len(ginfo_module)):
    l = ginfo_module[a]
    each_group = []
    for k in range(len(l)):
        j = l[k]
        j = [i for i in j if 3 <=  len(i) <= 20]
        j = [i for i in j if not any(c.isdigit() for c in i)]
        j = [i.lower() for i in j if i.isalpha()]
        j = remove_non_ascii_prinatble_from_list(j)   
        j = [lemmatizer.lemmatize(i, 'n') for i in j]
        j = [lemmatizer.lemmatize(i, 'v') for i in j]
        j = [i for i in j if i in model]
        each_group.append(j)
    clean_ginfo_module.append(each_group)

In [ ]:
# Flatten lists for modules
flat_ginfo_module = []
for module in clean_ginfo_module:
    flat_module = [item for sublist in module for item in sublist]
    flat_ginfo_module.append(flat_module)

In [ ]:
len(flat_ginfo_module)

In [ ]:
# Dictionary to store each module and their name
dict_module_ginfo = {}
for i in range(len(module_name)):
    dict_module_ginfo[module_name[i]] = clean_ginfo_module[i]

In [ ]:
# Count occurrence of each word in each module
from collections import Counter
count = []
for i in range(len(flat_ginfo_module)):
    j = flat_ginfo_module[i]
    k = Counter(item for item in j)
    count.append(k)

In [ ]:
# Store the most common words (top 100) in each module
most_common = []
for i in range(len(count)):
    j = count[i]
    k = j.most_common(100)
    most_common.append(k)

In [ ]:
# Further processing the most common words for each module
mostcommon_word = []
for i in range(len(most_common)):
    j = most_common[i]
    each = []
    for k in j:
        m = k[0]
        each.append(m)
    mostcommon_word.append(each)

In [ ]:
# Store in a dataframe
common_word = pd.DataFrame(mostcommon_word, index = module_name).transpose()

In [ ]:
# Output as csv
common_word.to_csv(r"C:\Users\lunad\Desktop\module_common_words_top100.csv")

In [ ]:
# Another way to sort the words in each module based on their frequency
dt = []
for i in range(len(count)):
    j = count[i]
    l = dict(j)
    l_sorted = {k: v for k, v in sorted(l.items(), key=lambda x: x[1], reverse = True)}
    dt.append(l_sorted)

In [ ]:
# Get how many words are there in each module
len_dt = []
for i in dt:
    l = len(i)
    len_dt.append(l)
    
len_dt

In [ ]:
# Top 1% based on total number of words
top_1per = []
for i in len_dt:
    l = int(i*0.01)
    top_1per.append(l)
top_1per

In [ ]:
m2 = list(dt[0].keys())[0:93]
m1 = list(dt[1].keys())[0:30]
m3 = list(dt[2].keys())[0:48]
m5 = list(dt[3].keys())[0:36]
m6 = list(dt[4].keys())[0:68]
m7 = list(dt[5].keys())[0:46]
m8 = list(dt[6].keys())[0:27]
m0 = list(dt[7].keys())[0:35]

In [ ]:
m = [m2,m1,m3,m5,m6,m7,m8,m0]

In [ ]:
#module_name = ["module2","module1","module3","module4","module5","module6","module7","module8","module0"]
modules = [module_2, module_1, module_3, module_4, module_5, module_6, module_7, module_8, module_0]

In [ ]:
# Top 10% of words in terms of their occurrence
le = []
for i in modules:
    l = len(i)
    l = int(l*0.1)
    le.append(l)

In [ ]:
m_2 = {key: dt[0][key] for key in dt[0] if dt[0][key] >= 70}
m_1 = {key: dt[1][key] for key in dt[1] if dt[1][key] >= 21}
m_3 = {key: dt[2][key] for key in dt[2] if dt[2][key] >= 27}
m_5 = {key: dt[3][key] for key in dt[3] if dt[3][key] >= 51}
m_6 = {key: dt[4][key] for key in dt[4] if dt[4][key] >= 45}
m_7 = {key: dt[5][key] for key in dt[5] if dt[5][key] >= 59}
m_8 = {key: dt[6][key] for key in dt[6] if dt[6][key] >= 18}
m_0 = {key: dt[7][key] for key in dt[7] if dt[7][key] >= 25}

In [ ]:
mm = [m_2,m_1,m_3,m_5,m_6,m_7,m_8,m_0]

In [ ]:
# Exxtract keys
keys = []
for i in range(len(mm)):
    j = mm[i]
    k = j.keys()
    l = list(k)
    keys.append(l)

# DTM for Module 2 (Biggest one)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
import nltk
from nltk.stem import WordNetLemmatizer 

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

mlb = MultiLabelBinarizer()
arrays = []
cla = []
for i in range(len(clean_ginfo_module)):
    data = clean_ginfo_module[i]
    array = mlb.fit_transform(data)
    classes = mlb.classes_
    arrays.append(array)
    cla.append(classes)

In [ ]:
df_m2 = pd.DataFrame(arrays[0], index = module_2, columns = cla[0])
#df_m2 = df_m2[m2]

In [ ]:
#df_m1 = pd.DataFrame(arrays[1], index = module_1, columns = cla[1])
df_m3 = pd.DataFrame(arrays[2], index = module_3, columns = cla[2])
df_m5 = pd.DataFrame(arrays[3], index = module_5, columns = cla[3])
df_m6 = pd.DataFrame(arrays[4], index = module_6, columns = cla[4])
df_m7 = pd.DataFrame(arrays[5], index = module_7, columns = cla[5])
df_m8 = pd.DataFrame(arrays[6], index = module_8, columns = cla[6])
df_m0 = pd.DataFrame(arrays[7], index = module_0, columns = cla[7])

In [ ]:
df_m3 = df_m3[m3]
df_m5 = df_m5[m5]
df_m6 = df_m6[m6]
df_m7 = df_m7[m7]
df_m8 = df_m8[m8]
df_m0 = df_m0[m0]

In [ ]:
df_m2 = pd.read_csv(r"C:\Users\lunad\Desktop\hai\dtm_m2.csv")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
import pandas as pd
import re

In [ ]:
# Run TF-IDF in the DTM where document refers to different modules
vect_m = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x)
tfidf_matrix_m = vect_m.fit_transform(flat_ginfo_module)
# Calcuate Euclidean distance between modules based on the tf-idf matrix
e_distance_m = euclidean_distances(tfidf_matrix_m)
tf_df_m = pd.DataFrame(tfidf_matrix_m.toarray(), columns = vect_m.get_feature_names())
e_df_m = pd.DataFrame(e_distance_m, index = module_name, columns = module_name)

In [ ]:
# PCA on modules: 8 to 4
import pandas as pd
import matplotlib.pyplot as plt
#from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%config InlineBackend.figure_format='retina'

# Standardize the data to have a mean of ~0 and a variance of 1
# X_std = StandardScaler().fit_transform(df_m2)
# Create a PCA instance: pca
pca_m = PCA(n_components=4)
principalComponents_m = pca_m.fit_transform(tf_df_m)
features_m = range(pca_m.n_components)

In [ ]:
plt.bar(features_m, pca_m.explained_variance_ratio_, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(features_m)

In [ ]:
PCA_components = pd.DataFrame(principalComponents)
plt.scatter(PCA_components[0], PCA_components[1], alpha=.1, color='black')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')

In [ ]:
# Run TF-IDF in the DTM where document refers to each group in a module
vect_g = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x)
tfidf_matrix_g = vect_g.fit_transform(clean_ginfo_module[2])
e_distance_g = euclidean_distances(tfidf_matrix)
tf_df_g = pd.DataFrame(tfidf_matrix_g.toarray(), columns = vect_g.get_feature_names())
e_df_g = pd.DataFrame(e_distance_g)

In [ ]:
# PCA on groups for each module
# Take module 2 as example
import pandas as pd
import matplotlib.pyplot as plt
#from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%config InlineBackend.figure_format='retina'

# Standardize the data to have a mean of ~0 and a variance of 1
# X_std = StandardScaler().fit_transform(df_m2)
# Create a PCA instance: pca
pca_m2 = PCA(n_components=10)
principalComponents_m2 = pca_m2.fit_transform(tf_df_g)
features_m2 = range(pca_m2.n_components)

In [ ]:
plt.bar(features_m2, pca_m2.explained_variance_ratio_, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(features_m2)

# MDS

In [ ]:
from sklearn.manifold import MDS
model = MDS(n_components=2, dissimilarity='euclidean', random_state=1)
out = model.fit_transform(e_distance_m)
results = model.fit(e_df_m.values)

In [ ]:
colorize = dict(c=e_distance_m[:, 0], cmap=plt.cm.get_cmap('rainbow', 5))
plt.scatter(out[:, 0], out[:, 1], **colorize)
plt.axis('equal')

In [ ]:
coords = results.embedding_
colorize = dict(c=e_distance_m[:, 0], cmap=plt.cm.get_cmap('rainbow', 8))
plt.subplots_adjust(bottom = 0.1)
plt.scatter(coords[:, 0], coords[:, 1], **colorize)

for label, x, y in zip(e_df_m.columns, coords[:, 0], coords[:, 1]):
    plt.annotate(
        label,
        xy = (x, y), 
        xytext = (-20, 20),
        textcoords = 'offset points'
    )
plt.show()